# 라이브러리 설치 및 로드

In [1]:
# %pip install -U bitsandbytes
# %pip install datasets -U
# %pip install -U git+https://github.com/huggingface/transformers.git
# %pip install -U git+https://github.com/huggingface/peft.git
# %pip install -U git+https://github.com/huggingface/accelerate.git
# %pip install pandas wandb matplotlib seaborn python-dotenv

In [2]:
# %pip install --upgrade transformers peft awq

In [1]:
import os
import torch
import pandas as pd
import bitsandbytes as bnb

from typing import Union
from functools import partial

from datasets import load_dataset
from huggingface_hub import login
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig, 
    Trainer, 
    TrainingArguments, 
    DataCollatorForSeq2Seq
)
from peft import (
    LoraConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)

In [2]:
from dotenv import load_dotenv

load_dotenv("../keys.env")
hf_token = os.getenv("HF_TOKEN_READ")

my_hf_key = hf_token
login(my_hf_key)

# 모델 path
model_path = "beomi/Llama-3-Open-Ko-8B" ## "meta-llama/Meta-Llama-3.1-8B-Instruct"

# 데이터 path
## "MarkrAI/KOpen-HQ-Hermes-2.5-60K"
## "DopeorNope/Ko-Optimize_Dataset"
## "Bingsu/ko_alpaca_data"
## "beomi/KoAlpaca-v1.1a"
data_path = "MarkrAI/KOpen-HQ-Hermes-2.5-60K"

# 토크나이저 및 데이터 준비

## Text Data 준비

In [ ]:
# dataset 다운
print(data_path)
data = load_dataset(data_path)

In [ ]:
df = pd.DataFrame(data['train'])
df.head()

- input : system prompt와 동일. 모델에게 적절한 행동을 요구.
- instruction : 수행했으면 하는 명령어.
- output : 원하는 답변.

In [ ]:
sample = df.iloc[0]
print(sample.keys())
print(f"Input : \n{sample['input']}\n")
print(f"Instruction : \n{sample['instruction']}\n")
print(f"Output : \n{sample['output']}")

# 토크나이저 준비

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

특수 토큰들을 출력해보면 PAD 토큰이 없음을 알 수 있다.

In [ ]:
tokenizer.special_tokens_map ## {'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>'}

In [ ]:
tokenizer.padding_side

In [9]:
# 토크나이저 세팅: QLoRA시 pad 토큰을 eos로 설정해주기
"""
llama3에서는 pad 토큰이 사용되지 않았는데 이러한 경우 eos 토큰을 pad 토큰으로 사용함.
단, transformers 라이브러리의 trainer를 활용할 때는 문제가 없으나 trl의 SFTTrainer를 사용하는 경우 eos 토큰을 생성하지 않고 무한히 단어를 생성해버리는 문제가 있음.
"""
bos = tokenizer.bos_token_id
eos = tokenizer.eos_token_id

# tokenizer.add_special_tokens({"pad_token" : "<|reserved_special_token_0|>"})
# tokenizer.padding_side = "right"

tokenizer.pad_token_id = eos
tokenizer.padding_side = "right"

# 학습 파라미터 설정

In [10]:
cut_off_len = 4098 ## 잘라서 사용할 sequence length
val_size = 0.005 ## 전체 데이터셋의 0.005%를 valid set으로 사용한다.
train_on_inputs = False ## 학습 단계에서 시작(질문의 첫번째 토큰)부터 끝(정답의 마지막 토큰)까지 모두 loss를 학습 -> context에 대한 이해도가 높아지게 된다.
add_eos_token = False ## tokenizing 단계에서 eos 토큰을 사용하지 않음.

# 템플릿 정의

input 컬럼 값이 있는 데이터와 없는 데이터 각각에 별도의 템플릿이 만들어지도록 구성함.

In [11]:
## 알파카 템플릿
template = {
    "prompt_input": "아래는 문제를 설명하는 지시사항과, 구체적인 답변을 방식을 요구하는 입력이 함께 있는 문장입니다. 이 요청에 대해 적절하게 답변해주세요.\n###입력:{input}\n###지시사항:{instruction}\n###답변:",
    "prompt_no_input": "아래는 문제를 설명하는 지시사항입니다. 이 요청에 대해 적절하게 답변해주세요.\n###지시사항:{instruction}\n###답변:"
}

In [12]:
def generate_prompt(
    instruction: str,
    template : dict,
    input: Union[None, str] = None,
    label: Union[None, str] = None,
    verbose: bool = False
) -> str:
    """
    주어진 instruction, input, output(label)을 사용하여 프롬프트를 생성하는 함수.

    Parameters:
    - instruction (str): 문제 설명 또는 지시사항.
    - template (dict): 입력이 있는 경우와 없는 경우의 템플릿을 포함한 딕셔너리.
    - input (str or None): 문제에 대한 구체적인 입력 (옵션).
    - label (str or None): 정답 또는 응답 (옵션).
    - verbose (bool): 생성된 프롬프트를 출력할지 여부.

    Returns:
    - str: 완성된 프롬프트.
    """
    if input:
        res = template["prompt_input"].format(instruction=instruction, input=input)
    else:
        res = template["prompt_no_input"].format(instruction=instruction)

    if label:
        res = f"{res}{label}"

    if verbose:
        print(res)

    return res


In [ ]:
print(generate_prompt(df.iloc[0]['instruction'], template, df.iloc[0]['input'], df.iloc[0]['output']))

In [ ]:
print(generate_prompt(df.iloc[1]['instruction'], template, df.iloc[1]['input'], df.iloc[1]['output']))

# 프롬프트 생성

In [15]:
def tokenize(prompt, add_eos_token=True):
   result = tokenizer(prompt, 
                      truncation=True, 
                      max_length=cut_off_len, 
                      padding=False, 
                      return_tensors=None)
   
   ## 토큰화된 값의 마지막이 eos_token_id와 같지 않고
   ## input_ids의 길이가 cut_off_len보다 작고
   ## add_eos_token이 True일 때 eos 토큰을 추가하고 attention_mask에 1을 추가함.
   if (result["input_ids"][-1] != tokenizer.eos_token_id
       and len(result["input_ids"]) < cut_off_len
       and add_eos_token
      ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

   result["labels"] = result["input_ids"].copy() ## input_ids를 label로 사용.(Auto Regressive)
   return result

In [ ]:
sample_prompt = generate_prompt(df.iloc[0]['instruction'], template, df.iloc[0]['input'], df.iloc[0]['output'])
print(sample_prompt)

In [ ]:
sample_prompt = tokenize(sample_prompt, False)
print(sample_prompt)

In [20]:
def generate_and_tokenize_prompt(data_point, template):
    ## 학습에 사용될 최종 프롬프트 생성.
    full_prompt = generate_prompt(
        instruction=data_point["instruction"],
        template=template,
        input=data_point["input"],
        label=data_point["output"]
    )
    
    ## 토크나이징
    tokenized_full_prompt = tokenize(full_prompt)
    
    ## train_on_inputs == False일 때 실행되어 정답(output)에 해당하는 토큰에서만 손실이 계산되게 된다.
    if not train_on_inputs:
        ## output을 제외한 instruction과 input만으로 프롬프트를 생성.
        user_prompt = generate_prompt(data_point["instruction"], template, data_point["input"])
        
        ## user_prompt를 토크나이징하며 eos 토큰을 추가하지 않음.
        ## 강의에서는 단순히 토큰 수를 하나 줄이기 위함으로 설명하는데 EOS 토큰이 추가되면, 모델은 instruction과 input만으로 문장이 끝났다고 판단하여, output에 대한 예측값을 생성하지 않을 위험이 있어서??
        tokenized_user_prompt = tokenize(user_prompt, add_eos_token=add_eos_token)
        user_prompt_len = len(tokenized_user_prompt["input_ids"])

        ## add_eos_token=True일 때 EOS 토큰 없이 토크나이징한 결과보다 1 길다.
        ## user_prompt_len은 instruction과 input로 구성된 프롬프트의 유효한 길이를 나타내기 때문에 길이 측정시 eos 토큰을 제외한 instruction + input의 길이만 계산해야한다.
        if add_eos_token:
            user_prompt_len -= 1

        ## instruction, input에 해당하는 부분에는 -100을 곱해서 학습을 하지 않게 하고, output 부분에서만 손실이 발생되도록 만든다.
        tokenized_full_prompt["labels"] = [-100] * user_prompt_len + tokenized_full_prompt["labels"][user_prompt_len:]

    return tokenized_full_prompt

# Validation set

In [ ]:
# Validation set
if val_size > 0:
    train_val = data["train"].train_test_split(test_size=val_size, shuffle=True, seed=42)
    
    # partial로 template 전달
    train_data = train_val["train"].shuffle().map(partial(generate_and_tokenize_prompt, template=template))
    val_data = train_val["test"].shuffle().map(partial(generate_and_tokenize_prompt, template=template))
else:
    train_data = data["train"].shuffle().map(partial(generate_and_tokenize_prompt, template=template))
    val_data = None

# Model 준비

In [22]:
# Quantization config 준비
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_storage=torch.bfloat16,
)

In [ ]:
# Model 로드 하기
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config = quantization_config,
    torch_dtype = torch.bfloat16,
    device_map = {"" : 0}
)

## For LoRA
# model = AutoModelForCausalLM.from_pretrained(
#     model_path,
#     device_map="auto"  # 장치 매핑 자동화
# )


In [ ]:
print(model.config.pad_token_id)

In [ ]:
## tokenizer.add_special_tokens({"pad_token" : "<|reserved_special_token_0|>"})
## tokenizer.padding_side = "right"
## 이렇게 pad 토큰을 별도로 추가해준 경우 model.config의 pad_token_id로 반영해줘야한다.
## eos 토큰을 pad 토큰으로 활용한 경우는 제외.

# model.config.pad_token_id = tokenizer.pad_token_id

In [25]:
## 양자화 학습 준비. LoRA 학습시 비적용. 지금은 QLoRA니까 적용.
model = prepare_model_for_kbit_training(model)

In [26]:
config = LoraConfig(
    r = 16, ## 랭크
    lora_alpha = 32, ## AB를 W에 얼마나 더해줄 것인지. 가중치.
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'], ## 어텐션 모듈에 Q, K, V에 대해 Adapter 추가.
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM"
)

In [27]:
## adapter를 추가할 선형 모듈들이 무엇이 있는지 탐색.
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])

  return list(lora_module_names)

print('Trainable targer module:',find_all_linear_names(model))

In [ ]:
print('Trainable targer module:',find_all_linear_names(model))

In [29]:
# QLoRA 준비
model = get_peft_model(model, config)

In [ ]:
model

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

# 파라미터 수 체크
print_trainable_parameters(model)

In [32]:
#Hyper parameter setting

output_dir='./llama_singleGPU-v1'

num_epochs = 1
micro_batch_size = 1
gradient_accumulation_steps = 16
warmup_steps = 10
learning_rate = 5e-7
group_by_length = False
optimizer = 'paged_adamw_8bit'

# adam 활용시
beta1 = 0.9
beta2 = 0.95

lr_scheduler = 'cosine'
logging_steps = 1

use_wandb = True
wandb_run_name = 'Single_GPU_Optim'

use_fp16 = False
use_bf_16 = True
evaluation_strategy = 'steps'
eval_steps = 50
save_steps = 50
save_strategy = 'steps'

In [33]:
model.gradient_checkpointing_enable()

In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=TrainingArguments(
    per_device_train_batch_size = micro_batch_size,
    per_device_eval_batch_size = micro_batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    warmup_steps = warmup_steps,
    num_train_epochs = num_epochs,
    learning_rate = learning_rate,
    adam_beta1 = beta1, # adam 활용할때 사용
    adam_beta2 = beta2, # adam 활용할때 사용
    fp16 = use_fp16,
    bf16 = use_bf_16,
    logging_steps = logging_steps,
    optim = optimizer,
    evaluation_strategy = evaluation_strategy if val_size > 0 else "no",
    save_strategy="steps",  #스텝기준으로 save
    eval_steps = eval_steps if val_size > 0 else None,
    save_steps = save_steps,
    lr_scheduler_type=lr_scheduler,
    output_dir = output_dir,
    #save_total_limit = 4,
    load_best_model_at_end = True if val_size > 0 else False ,
    group_by_length=group_by_length,
    report_to="wandb" if use_wandb else None,
    run_name=wandb_run_name if use_wandb else None,
    ),
    data_collator=DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
        ),
    )

In [ ]:
model.config.use_cache = False


trainer.train()

In [ ]:
trainer.save_model()
tokenizer.save_pretrained(output_dir)

In [ ]:
## Trainer가 저장하는 것은 전체 모델 + 어뎁터가 아니라 어뎁터만 저장하는 것이기 때문에 별도로 base_model과 학습된 어뎁터를 병합하는 과정이 필요하다.

base_model = AutoModelForCausalLM.from_pretrained(model_path, token=my_hf_key)

merged_model = PeftModel.from_pretrained(base_model, output_dir)
merged_model = merged_model.merge_and_unload()

# merged_model.push_to_hub("DoporNope/Single_GPU_Llama3-8B")
# tokenizer.push_to_hub("DopeorNope/Single_GPU_Llama3-8B")